#### Target variable : risk decision to 3 levels
#### Feature: Subject (after sentence embedding) only 
#### Model: CatBoost, XGBoost, Randomzied serach for XGboost

In [2]:
import pandas as pd

df=pd.read_csv("/content/rasff_new2.csv")

In [3]:
# ----------------------
# Load and prepare data
# ----------------------
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# ----------------------
# Time-based split (80/20)
# ----------------------
cutoff_index = int(len(df) * 0.8)
cutoff_date = df.iloc[cutoff_index]['date']

train_df = df[df['date'] <= cutoff_date]
test_df  = df[df['date'] > cutoff_date]

In [4]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 2.5 MB/s eta 0:00:00


In [5]:
from category_encoders import CatBoostEncoder
from sentence_transformers import SentenceTransformer

In [6]:
# ----------------------
# Sentence Transformer Embedding
# ----------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

train_embeddings = model.encode(train_df['subject'].tolist(), show_progress_bar=True)
test_embeddings  = model.encode(test_df['subject'].tolist(), show_progress_bar=True)

# Add embedding vectors back into dataframe (384 dims)
emb_cols = [f"sub_emb_{i}" for i in range(train_embeddings.shape[1])]

train_df[emb_cols] = train_embeddings
test_df[emb_cols]  = test_embeddings

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/685 [00:00<?, ?it/s]

Batches:   0%|          | 0/172 [00:00<?, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/tmp/ipython-input-3329622934.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[emb_cols] = train_embeddings
/tmp/ipython-input-3329622934.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[emb_cols] = train_embeddings
/tmp/ipython-input-3329622934.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [8]:
train_df[emb_cols]

,sub_emb_0,sub_emb_1,sub_emb_2,sub_emb_3,sub_emb_4,sub_emb_5,sub_emb_6,sub_emb_7,sub_emb_8,sub_emb_9,...,sub_emb_374,sub_emb_375,sub_emb_376,sub_emb_377,sub_emb_378,sub_emb_379,sub_emb_380,sub_emb_381,sub_emb_382,sub_emb_383
27396,0.087586,0.010154,0.006940,0.082257,0.064434,0.036325,-0.032588,0.008418,0.004947,-0.002062,...,-0.034038,0.037460,0.114971,-0.024915,0.061022,-0.059150,-0.016792,-0.032665,0.111988,0.015402
27395,0.087587,0.010154,0.006940,0.082257,0.064434,0.036325,-0.032588,0.008418,0.004947,-0.002062,...,-0.034038,0.037460,0.114971,-0.024915,0.061022,-0.059150,-0.016792,-0.032665,0.111988,0.015402
27394,0.008577,0.029172,-0.098910,0.072603,0.011805,-0.010723,-0.032932,0.054643,0.001715,-0.118847,...,-0.027457,0.029665,0.107867,0.056153,0.028187,-0.051156,0.001054,0.012128,0.117444,0.010728
27393,0.037681,-0.002600,-0.031736,0.023088,0.028602,-0.012209,-0.000027,-0.035374,0.014961,-0.055146,...,-0.097944,-0.013098,0.045925,-0.016163,0.041347,0.018233,0.071770,0.018242,-0.011143,0.035854
27392,-0.049578,0.022277,0.019572,0.040566,0.145016,-0.017994,-0.019010,-0.075383,-0.012035,0.041207,...,-0.100262,-0.041913,0.003566,0.134337,0.063356,-0.033248,0.006672,-0.017443,0.073693,0.015101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,0.013757,-0.031772,-0.051145,-0.015934,-0.018717,-0.022059,-0.034356,0.105511,0.050685,-0.000033,...,-0.055427,0.022467,0.034420,-0.089294,0.033445,-0.043342,0.006678,-0.003936,0.038431,0.057685
5482,0.047038,-0.014314,-0.121896,0.008590,0.034609,-0.008880,-0.004360,0.042058,0.009392,-0.037499,...,-0.039020,-0.042285,0.110709,0.061534,-0.015145,-0.057617,-0.039785,0.058867,0.086120,0.011481
5481,-0.084203,0.068677,0.011133,-0.081151,-0.058607,0.028688,0.010080,0.054687,-0.038090,-0.098137,...,-0.049851,0.041745,-0.013039,0.007178,-0.008795,-0.007229,-0.018705,-0.020237,0.029319,-0.035429
5480,0.018501,0.028768,-0.041562,0.024362,0.035141,0.063047,0.046218,0.088342,0.025676,-0.097672,...,-0.093018,0.001194,0.023885,0.031591,0.054121,-0.024105,0.011932,-0.061453,0.026549,-0.027947


In [7]:
# ----------------------------
# Recode risk_decision into 3 classes
# ----------------------------
def recode_risk(risk):
    if risk in ['no risk', 'not serious']:
        return 0   # Low risk
    elif risk in ['potential risk', 'undecided', 'potentially serious']:
        return 1   # Medium / potential risk
    elif risk == 'serious':
        return 2   # High risk
    else:
        return -1  # Safety net for unexpected values

# Apply to both train and test
train_df['risk_decision_3class'] = train_df['risk_decision'].apply(recode_risk)
test_df['risk_decision_3class']  = test_df['risk_decision'].apply(recode_risk)

# Optional: check distribution
print(train_df['risk_decision_3class'].value_counts())
print(test_df['risk_decision_3class'].value_counts())

# Update target variable
target_col = 'risk_decision_3class'

risk_decision_3class
2    12204
1     5967
0     3747
Name: count, dtype: int64
risk_decision_3class
2    2552
1    2123
0     804
Name: count, dtype: int64


/tmp/ipython-input-2619610551.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['risk_decision_3class'] = train_df['risk_decision'].apply(recode_risk)
/tmp/ipython-input-2619610551.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['risk_decision_3class'] = train_df['risk_decision'].apply(recode_risk)
/tmp/ipython-input-2619610551.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [10]:
embedding_only_train = train_df[emb_cols]
embedding_only_test  = test_df[emb_cols]

In [11]:
print(len(embedding_only_train.columns))

384


In [12]:
print(len(embedding_only_test.columns))

384


In [ ]:
# Identify features
# train_df and test_df already have embeddings + encoded features
# Define target column (After mapping)
target_col = 'risk_decision_3class'


In [21]:
# Prepare data
X_train = train_df[emb_cols].values
y_train = train_df[target_col].values
X_test  = test_df[emb_cols].values
y_test  = test_df[target_col].values

In [15]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test, label=y_test)

# Define multi-class parameters
num_classes = len(train_df[target_col].unique())
params = {
    'objective': 'multi:softmax',  # or 'multi:softprob' for probabilities
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'eta': 0.03,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Train
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=50,
    verbose_eval=50
)

# Predict
y_pred = bst.predict(dtest)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[0]	train-mlogloss:1.02126	test-mlogloss:1.04863
[50]	train-mlogloss:0.72467	test-mlogloss:0.89058
[100]	train-mlogloss:0.60287	test-mlogloss:0.84975
[150]	train-mlogloss:0.52819	test-mlogloss:0.83170
[200]	train-mlogloss:0.47425	test-mlogloss:0.82029
[250]	train-mlogloss:0.43156	test-mlogloss:0.81272
[300]	train-mlogloss:0.39613	test-mlogloss:0.80737
[350]	train-mlogloss:0.36485	test-mlogloss:0.80314
[400]	train-mlogloss:0.33807	test-mlogloss:0.79983
[450]	train-mlogloss:0.31463	test-mlogloss:0.79799
[500]	train-mlogloss:0.29281	test-mlogloss:0.79635
[550]	train-mlogloss:0.27338	test-mlogloss:0.79540
[600]	train-mlogloss:0.25613	test-mlogloss:0.79565
[601]	train-mlogloss:0.25582	test-mlogloss:0.79574
Accuracy: 0.6329622193830992
              precision    recall  f1-score   support

           0       0.58      0.43      0.50       804
           1       0.64      0.42      0.51      2123
           2       0.64      0.88      0.74      2552

    accuracy                           0.6

In [22]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.9 MB/s eta 0:00:00


In [23]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    loss_function='MultiClass',
    learning_rate=0.03,
    depth=6,
    iterations=1500,
    eval_metric='TotalF1',
    verbose=False
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)
y_pred = model.predict(X_test)

0:	learn: 0.5316696	test: 0.4648305	best: 0.4648305 (0)	total: 536ms	remaining: 13m 24s
100:	learn: 0.6122074	test: 0.4937127	best: 0.4937127 (100)	total: 42.9s	remaining: 9m 54s
200:	learn: 0.6724786	test: 0.5307461	best: 0.5307461 (200)	total: 1m 15s	remaining: 8m 6s
300:	learn: 0.7056670	test: 0.5557842	best: 0.5580494 (285)	total: 1m 45s	remaining: 6m 59s
400:	learn: 0.7328523	test: 0.5701761	best: 0.5709434 (397)	total: 2m 17s	remaining: 6m 16s
500:	learn: 0.7535220	test: 0.5782027	best: 0.5791398 (487)	total: 2m 47s	remaining: 5m 33s
600:	learn: 0.7720854	test: 0.5835295	best: 0.5844224 (593)	total: 3m 16s	remaining: 4m 54s
700:	learn: 0.7867362	test: 0.5859366	best: 0.5863214 (647)	total: 3m 46s	remaining: 4m 18s
800:	learn: 0.7975667	test: 0.5868427	best: 0.5876829 (725)	total: 4m 15s	remaining: 3m 43s
900:	learn: 0.8095012	test: 0.5933789	best: 0.5934380 (893)	total: 4m 46s	remaining: 3m 10s
1000:	learn: 0.8184970	test: 0.5977099	best: 0.5981481 (998)	total: 5m 16s	remaining: 

In [24]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6285818580032853
              precision    recall  f1-score   support

           0       0.58      0.42      0.49       804
           1       0.64      0.41      0.50      2123
           2       0.63      0.87      0.73      2552

    accuracy                           0.63      5479
   macro avg       0.62      0.57      0.57      5479
weighted avg       0.63      0.63      0.61      5479



In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Prepare data
X_train = train_df[emb_cols].values
y_train = train_df[target_col].values
X_test  = test_df[emb_cols].values
y_test  = test_df[target_col].values

# Scaling helps logistic regression a lot
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Best-performing solver for >100 features
clf = LogisticRegression(
    multi_class='multinomial',
    class_weight='balanced',   # important due to imbalance
    solver='lbfgs',
    max_iter=2000,
    random_state=42
)

clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.5926263916773133
              precision    recall  f1-score   support

           0      0.379     0.617     0.470       804
           1      0.590     0.472     0.524      2123
           2      0.707     0.686     0.696      2552

    accuracy                          0.593      5479
   macro avg      0.559     0.591     0.563      5479
weighted avg      0.614     0.593     0.596      5479



In [26]:
from sklearn.svm import LinearSVC

# Prepare data
X_train = train_df[emb_cols].values
y_train = train_df[target_col].values
X_test  = test_df[emb_cols].values
y_test  = test_df[target_col].values

# Scaling helps logistic regression a lot
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

clf = LinearSVC(class_weight='balanced')
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)


print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

Accuracy: 0.6054024457017704
              precision    recall  f1-score   support

           0      0.415     0.565     0.478       804
           1      0.617     0.421     0.501      2123
           2      0.671     0.772     0.718      2552

    accuracy                          0.605      5479
   macro avg      0.568     0.586     0.566      5479
weighted avg      0.612     0.605     0.598      5479



In [28]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import randint, uniform
import numpy as np

# ---------------------------
# Prepare data
# ---------------------------
X_train = train_df[emb_cols].values
y_train = train_df[target_col].values
X_test  = test_df[emb_cols].values
y_test  = test_df[target_col].values

num_classes = len(np.unique(y_train))

# ---------------------------
# Define Base Model
# ---------------------------
xgb = XGBClassifier(
    objective='multi:softprob',     # softer boundaries, better performance
    num_class=num_classes,
    eval_metric='mlogloss',
    tree_method='hist',             # fast for dense embeddings
    random_state=42,
    use_label_encoder=False
)

# ---------------------------
# Randomized Search Parameter Space
# ---------------------------
param_dist = {
    "learning_rate": uniform(0.01, 0.2),      # loc=0.01, scale=0.2 → range: 0.01–0.21
    "max_depth": randint(3, 10),
    "min_child_weight": randint(1, 10),
    "subsample": uniform(0.6, 0.4),           # range: 0.6–1.0
    "colsample_bytree": uniform(0.6, 0.4),    # range: 0.6–1.0
    "n_estimators": randint(300, 1200),
    "gamma": uniform(0, 5),
    "reg_alpha": uniform(0, 1),
    "reg_lambda": uniform(0.5, 2),
}

# ---------------------------
# RandomizedSearchCV
# ---------------------------
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,                     # try 30 random combinations
    scoring="accuracy",
    cv=3,
    verbose=2,
    n_jobs=-1,                     # use all CPU cores
    random_state=42,
)

print("Running RandomizedSearchCV...")
random_search.fit(X_train, y_train)

print("Best parameters found:")
print(random_search.best_params_)

# ---------------------------
# Evaluate Best Model
# ---------------------------
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

Running RandomizedSearchCV...
Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [13:55:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters found:
{'colsample_bytree': np.float64(0.9895022075365837), 'gamma': np.float64(1.1638567021515211), 'learning_rate': np.float64(0.02812128690656416), 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 1175, 'reg_alpha': np.float64(0.5142344384136116), 'reg_lambda': np.float64(1.684829137724085), 'subsample': np.float64(0.6185801650879991)}

Accuracy: 0.6311370688081767
              precision    recall  f1-score   support

           0      0.579     0.434     0.496       804
           1      0.640     0.410     0.500      2123
           2      0.637     0.877     0.738      2552

    accuracy                          0.631      5479
   macro avg      0.618     0.574     0.578      5479
weighted avg      0.629     0.631     0.610      5479

